In [13]:
!pip install keras_cv image_classifiers
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from classification_models.keras import Classifiers
from keras.datasets import cifar100
from keras import Sequential
from keras import Input
from keras.layers import UpSampling2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import RandomFlip
from keras.layers import RandomTranslation
from keras.layers import RandomRotation
from keras.layers import RandomBrightness
from keras.layers import RandomContrast
from keras.layers import RandomZoom
from keras.layers import GlobalAveragePooling2D
from keras_cv.layers import RandomShear
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.models import Model
import keras_cv
import re
import os
import datetime

In [14]:
n_epoch = 1000 # Il y a le early stopping
batch_size = 400

In [15]:
(X_train, y_train), (X_test, y_test) = cifar100.load_data()
X_train = np.float64(X_train)/255
X_test = np.float64(X_test)/255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_classes = 100

In [17]:
modele_enseignant = tf.keras.models.load_model("/content/drive/MyDrive/projet_cnam/modele_745.keras")

In [18]:
y_train_enseignant = modele_enseignant.predict(X_train)

1563/1563 [==============================] - 31s 17ms/step


In [19]:
ResNet18, preprocess_input = Classifiers.get('resnet18')
resnet = ResNet18((224, 224, 3), weights='imagenet', include_top=False)
resnet_output = GlobalAveragePooling2D()(resnet.output)
resnet = Model(inputs=resnet.input, outputs=resnet_output)

resnet.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 data (InputLayer)           [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 bn_data (BatchNormalizatio  (None, 224, 224, 3)          9         ['data[0][0]']                
 n)                                                                                               
                                                                                                  
 zero_padding2d_18 (ZeroPad  (None, 230, 230, 3)          0         ['bn_data[0][0]']             
 ding2D)                                                                                          
                                                                                            

In [40]:
model = Sequential([
    UpSampling2D(size=(7, 7)),
    RandomFlip("horizontal"),
    RandomTranslation(0.2,0.2),
    RandomRotation(0.2),
    RandomZoom(0.2),
    RandomShear(0.2),
    RandomBrightness(0.2,value_range=(0,1)),
    RandomContrast(0.2),
    resnet,
    Dropout(0.25),
    Dense(256, activation="sigmoid", kernel_regularizer = tf.keras.regularizers.L1(0.001)),
    Dropout(0.5),
    Dense(num_classes, activation="softmax", kernel_regularizer = tf.keras.regularizers.L2(0.001) )])

In [41]:
model.compile(optimizer='adam', loss='kl_divergence', metrics=['accuracy'])
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train_enseignant, epochs=n_epoch, batch_size=batch_size, validation_split = 0.1,callbacks = early_stopping_callback)
model.save("/root/modele_enseigne.keras")

Epoch 1/1000
113/113 [==============================] - 40s 265ms/step - loss: 5.3630 - accuracy: 0.5674 - val_loss: 2.6600 - val_accuracy: 0.6406
Epoch 2/1000
113/113 [==============================] - 29s 259ms/step - loss: 2.0633 - accuracy: 0.7009 - val_loss: 1.7448 - val_accuracy: 0.6542
Epoch 3/1000
113/113 [==============================] - 29s 259ms/step - loss: 1.7098 - accuracy: 0.7152 - val_loss: 1.5873 - val_accuracy: 0.6694
Epoch 4/1000
113/113 [==============================] - 29s 259ms/step - loss: 1.6005 - accuracy: 0.7216 - val_loss: 1.5497 - val_accuracy: 0.6704
Epoch 5/1000
113/113 [==============================] - 29s 259ms/step - loss: 1.5250 - accuracy: 0.7345 - val_loss: 1.4691 - val_accuracy: 0.6820
Epoch 6/1000
113/113 [==============================] - 29s 259ms/step - loss: 1.4685 - accuracy: 0.7401 - val_loss: 1.7352 - val_accuracy: 0.6024
Epoch 7/1000
113/113 [==============================] - 29s 259ms/step - loss: 1.4116 - accuracy: 0.7501 - val_loss: 1